Rzeczy do (ewentualnego) poprawienia/obsłużenia:
- zamiast zwykłego multigrafu, zrobić multigraf skierowany, bo traktujemy loty w jedną strone jako loty w obie naraz *//poniekąd obsłużone*
- zawsze wyświetlana jest data wylotu. Oznacza to, że jeżeli przylot ma miejsce nastęonego dnia, ma on miejsce dzień później niż data w `arrival time` *//obsłużone*

In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import queue
import datetime
import time
import random


def get_price(dist):
    return round(np.random.normal(0.20, 0.07) * dist, 2)

def parse_date(year, month, day, time):
    return datetime.datetime(year, month, day, int('{:04d}'.format(time)[:2]), int('{:04d}'.format(time)[2:]))

COLS = ['YEAR', 'MONTH', 'DAY', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DISTANCE', 'SCHEDULED_TIME', 'SCHEDULED_ARRIVAL']
N = 100000 #max = 5819079

flights = pd.read_csv("data/flights.csv")[COLS]
flights = flights.head(N)
flights = flights[flights['SCHEDULED_TIME'].notna()]
flights['PRICE'] = flights['DISTANCE'].apply(lambda x: get_price(x))
# print(flights.head(10))

MG = nx.MultiGraph()

for idx, row in flights.iterrows():
    departure_time = parse_date(row['YEAR'], row['MONTH'], row['DAY'], row['SCHEDULED_DEPARTURE'])
    arrival_time = parse_date(row['YEAR'], row['MONTH'], row['DAY'], row['SCHEDULED_ARRIVAL'])
    if arrival_time < departure_time:
        arrival_time += datetime.timedelta(days=1)

    MG.add_edge(row['ORIGIN_AIRPORT'], row['DESTINATION_AIRPORT'], origin=row['ORIGIN_AIRPORT'], destination=row['DESTINATION_AIRPORT'], departure_time=departure_time, arrival_time=arrival_time, flight_time=row['SCHEDULED_TIME'], price=row['PRICE'], pheromone_level=0, pheromone_update_time=0)

In [ ]:
ANTS_NUMBER = 100
ANTS_SPAWN_ITERS = 10
CONNECTION_SAMPLES = 3

DIRECT_CONNECTION_IMPACT = 0.8
TIME_IMPACT = 0.4
COST_IMPACT = 0.2
PHEROMONE_IMPACT = 0.4

PHEROMONE_UPDATE = 0.5
PHEROMONE_UPDATING_TIME = 1000

COST_FUN_TIME_IMP = 0.5

In [ ]:
def clean_pheromones(mg):
    for airport_1 in mg.nodes:
        for airport_2 in mg.adj[airport_1]:
            for flight in mg.adj[airport_1][airport_2]:
                mg.adj[airport_1][airport_2][flight]['pheromone_level'] = 0
                mg.adj[airport_1][airport_2][flight]['pheromone_update_time'] = 0

In [ ]:
class Ant:
    def __init__(self, curr_time, curr_airport):
        self.curr_time = curr_time
        self.curr_airport = curr_airport
        self.mode = 'NORMAL'

        self.curr_trav_cost = 0
        self.curr_conn_numb = 0
        self.path = [curr_airport]
        
    def __lt__ (self, other):
        if self.mode != other.mode:
            return self.mode < other.mode
        return self.curr_time < other.curr_time
        
    def __gt__ (self, other):
        if self.mode != other.mode:
            return self.mode > other.mode
        return self.curr_time > other.curr_time

    def __eq__ (self, other):
        return self.mode == other.mode and self.curr_time == other.curr_time

    def __ne__ (self, other):
        return not self.__eq__(other)
    
    def print_ant(self):
        print(f'\ntravel cost: {self.curr_trav_cost}\nconnections number: {self.curr_conn_numb}\nflights:\n {self.path}\n')
        
    def update(self, next_flight):
        self.curr_time = next_flight[2]['arrival_time']
        self.curr_airport = next_flight[0]
        self.curr_trav_cost += next_flight[2]['price']
        self.curr_conn_numb += 1
        self.path.append((next_flight[0], next_flight[2]['departure_time'], next_flight[2]['arrival_time']))

In [ ]:
class AntColonyAlgorithm:
    def __init__(self, flights, origin, destination, min_time, max_time, min_conn_time, max_conn_numb, max_price):
        self.flights = flights
        self.origin = origin
        self.destination = destination
        self.min_time = min_time
        self.max_time = max_time
        self.min_conn_time = min_conn_time
        self.max_conn_numb = max_conn_numb
        self.max_price = max_price

        self.global_time = 0
        self.events = queue.PriorityQueue()

    ###
    def _init_ants(self):
        time_available_min = (self.max_time - self.min_time).total_seconds() // 60

        ants_spawn_gap = time_available_min // ANTS_SPAWN_ITERS
        for i in range(ANTS_SPAWN_ITERS):
            for j in range(ANTS_NUMBER // ANTS_SPAWN_ITERS):
                self.events.put((i * ants_spawn_gap, Ant(self.min_time + datetime.timedelta(minutes=i * ants_spawn_gap), self.origin)))

    ###
    def _run_next_event(self, disp_mess, get_results=False, results=None):
        self.global_time, curr_ant = self.events.get()
        available_flights = self._find_available_flights(curr_ant)
        self._update_pheromones(available_flights)
        next_flight = self._choose_flight(available_flights, curr_ant)
        self._make_next_flight(next_flight, curr_ant, disp_mess, get_results, results)

    ###
    def _find_available_flights(self, curr_ant):
        all_flights = self.flights[curr_ant.curr_airport]
        available_flights = []

        for airport in all_flights:
            flights_added = 0
            if curr_ant.mode == 'NORMAL':
                start, end, order = 0, len(self.flights.adj[curr_ant.curr_airport][airport]), 1
            else: # curr_ant.mode == 'RETURN'
                start, end, order = len(self.flights.adj[curr_ant.curr_airport][airport]) - 1, -1, -1

            for flight_idx in range(start, end, order):
                flight = self.flights.adj[curr_ant.curr_airport][airport][flight_idx]
                if self._is_accessible_flight(curr_ant, flight):
                    available_flights.append((airport, flight_idx, flight))
                    flights_added += 1
                    if flights_added == CONNECTION_SAMPLES:
                        break

        return available_flights

    def _is_accessible_flight(self, curr_ant, flight):
        if curr_ant.mode == 'NORMAL':
            good_airport = curr_ant.curr_airport == flight['origin']
            good_departure = curr_ant.curr_time + datetime.timedelta(minutes=self.min_conn_time) <= flight['departure_time']
            good_arrival = flight['arrival_time'] <= self.max_time
        else: # curr_ant.mode == 'RETURN'
            good_airport = curr_ant.curr_airport == flight['destination']
            good_arrival = curr_ant.curr_time - datetime.timedelta(minutes=self.min_conn_time) >= flight['arrival_time']
            good_departure = flight['departure_time'] >= self.min_time

        good_cost = curr_ant.curr_trav_cost + flight['price'] <= self.max_price
        return good_airport and good_departure and good_arrival and good_cost

    ###
    def _update_pheromones(self, available_flights):
        for airport, index, flight in available_flights:
            time_gap = self.global_time - flight['pheromone_update_time']
            flight['pheromone_level'] = flight['pheromone_level'] * PHEROMONE_UPDATE ** (time_gap // PHEROMONE_UPDATING_TIME)
            flight['pheromone_update_time'] = self.global_time

    ###
    def _choose_flight(self, flights, curr_ant):
        if len(flights) == 0:
            return None

        for airport, index, flight in flights:
            if curr_ant.mode == 'NORMAL' and airport == self.destination or curr_ant.mode == 'RETURN' and airport == self.origin:
                if random.random() < DIRECT_CONNECTION_IMPACT:
                    flight['pheromone_level'] += 1
                    return airport, index, flight

        waiting_times, prices, pheromones = np.empty(len(flights)), np.empty(len(flights)), np.empty(len(flights))

        min_time = curr_ant.curr_time + datetime.timedelta(minutes=self.min_conn_time) if curr_ant.mode == 'NORMAL' else None
        max_time = curr_ant.curr_time - datetime.timedelta(minutes=self.min_conn_time) if curr_ant.mode == 'RETURN' else None

        for index, (airport, flight_index, flight) in enumerate(flights):
            if curr_ant.mode == 'NORMAL':
                waiting_times[index] = (flight['departure_time'] - min_time).total_seconds() // 60
            if curr_ant.mode == 'RETURN':
                waiting_times[index] = (max_time - flight['arrival_time']).total_seconds() // 60
            prices[index] = flight['price']
            pheromones[index] = flight['pheromone_level']

        time_coeffs = (1 - np.nan_to_num(waiting_times / np.max(waiting_times))) ** 3
        prices_coeffs = (1 - np.nan_to_num(prices / np.max(prices))) ** 3
        pheromones_coeffs = np.nan_to_num(pheromones / np.max(pheromones))
        combined_params = time_coeffs * TIME_IMPACT + prices_coeffs  * COST_IMPACT + pheromones_coeffs * PHEROMONE_IMPACT

        flight_pos = random.uniform(0, np.sum(combined_params))
        for flight, params in zip(flights, combined_params):
            if params >= flight_pos:
                flight[2]['pheromone_level'] += 1
                return flight
            flight_pos -= params

    ###
    def _make_next_flight(self, next_flight, curr_ant, disp_mess, get_results, results):
        if next_flight is None or curr_ant.curr_conn_numb == self.max_conn_numb:
            self._kill_ant(curr_ant, disp_mess)
            self._spawn_ant()

        else:
            if curr_ant.mode == 'NORMAL':
                time_diff = (next_flight[2]['arrival_time'] - curr_ant.curr_time).total_seconds() // 60
            else: #curr_ant.mode == 'RETURN'
                time_diff = (curr_ant.curr_time - next_flight[2]['departure_time']).total_seconds() // 60
            new_time = self.global_time + time_diff

            curr_ant.update(next_flight)
            if curr_ant.mode == 'NORMAL' and next_flight[0] == self.destination:
                if disp_mess:
                    print('ANT REACHED DESTINATION')
                    curr_ant.print_ant()
                if get_results:
                    results.append((curr_ant.curr_trav_cost, curr_ant.curr_conn_numb, curr_ant.path))
                    self._spawn_ant()
                else:
                    curr_ant.curr_trav_cost = 0
                    curr_ant.mode = 'RETURN'
                    self.events.put((new_time, curr_ant))
            elif curr_ant.mode == 'RETURN' and next_flight[0] == self.origin:
                if disp_mess:
                    print('ANT REACHED DESTINATION AND CAME BACK')
                    curr_ant.print_ant()
                self._spawn_ant()
            else:
                self.events.put((new_time, curr_ant))

    def _kill_ant(self, curr_ant, disp_mess):
        if disp_mess:
            print('ANT DIED')
            curr_ant.print_ant()

    def _spawn_ant(self):
        time_available_min = (self.max_time - self.min_time).total_seconds() // 60
        ant_spawn_gap = random.randint(0, time_available_min)
        self.events.put((self.global_time + 1, Ant(self.min_time + datetime.timedelta(minutes=ant_spawn_gap), self.origin)))

    ###
    def _find_best_result(self, results):
        costs, total_times = np.empty(len(results)), np.empty(len(results))

        for index, result in enumerate(results):
            costs[index] = result[0]
            total_times[index] = (result[2][-1][2] - result[2][1][1]).total_seconds() // 60

        costs = np.nan_to_num(costs / np.max(costs))
        total_times = np.nan_to_num(total_times / np.max(total_times))
        full_params = total_times * COST_FUN_TIME_IMP + costs * (1 - COST_FUN_TIME_IMP)

        results_params = [(result, params) for result, params in zip(results, full_params)]
        results_params.sort(key=lambda x:x[1])
        return results_params[0][0]

    ###
    def run(self, mode, time_iters_numb=None, results_sample=None, disp_mess=True):
        self._init_ants()

        if mode == 'TIME':
            start_time = time.time()
            counter = 0
            while (counter % 100 != 0 or time.time() - start_time < time_iters_numb):
                self._run_next_event(disp_mess)
                counter += 1

        if mode == 'ITERS':
            counter = 0
            while counter < time_iters_numb:
                self._run_next_event(disp_mess)
                counter += 1

        if mode == 'RESULT':
            results = []
            while len(results) < results_sample:
                self._run_next_event(disp_mess, True, results)
            return self._find_best_result(results)


In [ ]:
clean_pheromones(MG)
simulation = AntColonyAlgorithm(MG, 'LAS', 'JAX', datetime.datetime(2015, 1, 1, 8, 0, 0), datetime.datetime(2015, 1, 8, 8, 0, 0), 90, 5, 5000)
simulation.run('ITERS', time_iters_numb=1000, disp_mess=False)
print(simulation.run('RESULT', results_sample=100, disp_mess=False))